In [45]:
pip install scikit-fuzzy


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [46]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

Generem dades artificials que de moment no tenim les oficials

In [47]:
import pandas as pd
import numpy as np

# Llista dels 51 estats dels EUA (noms complets)
estats_usa = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware",
    "District of Columbia", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa",
    "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota",
    "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico",
    "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island",
    "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington",
    "West Virginia", "Wisconsin", "Wyoming"
]
# Genera dades aleatòries realistes (ajustables)
np.random.seed(42)  # Per reprodibilitat

# Genera valors aleatoris per hospitalitzats, morts i contagiats
df_epidemiologic = pd.DataFrame({
    "state": estats_usa,
    "hospitalitzats": np.random.randint(50, 2000, size=51),  # Exemple: entre 50 i 2000
    "morts": np.random.randint(5, 500, size=51),            # Exemple: entre 5 i 500
    "contagiats": np.random.randint(1000, 50000, size=51)   # Exemple: entre 1000 i 50000
})

# Mostra les primeres files
print(df_epidemiologic.head())

        state  hospitalitzats  morts  contagiats
0     Alabama            1176     55       34763
1      Alaska            1509    368       33606
2     Arizona             910     59       12534
3    Arkansas            1344    248       30127
4  California            1180    324       41397


In [48]:
import pandas as pd

df_demografic_original = pd.read_csv("../data/preprocessed/dataMatrix/static_stateMatrix.csv")
columnes = [
    "state", 
    "population_state",
    "pop_density_state",
    "bedsTotal",
    "pop_0-9", "pop_10-19", "pop_20-29", "pop_30-39",
    "pop_40-49", "pop_50-59", "pop_60-69", "pop_70-79", "pop_80+"
]
df_demografic = df_demografic_original[columnes]

Fusionem per tenir dades població (x edad ens poden servir també x tenir zones amb molta gent gran)

In [49]:
# Fusiona els dos DataFrames per la columna "state"
df = pd.merge(
    df_demografic, 
    df_epidemiologic, 
    on="state", 
    how="inner" 
)

# Mostra les primeres files (opcional)
print(df.head())

        state  population_state  pop_density_state  bedsTotal   pop_0-9  \
0     Alabama           4887871          96.509389        3.1  0.120933   
1      Alaska            737438           1.291523        2.2  0.142617   
2     Arizona           7171646          63.135855        1.9  0.123774   
3    Arkansas           3013825          57.919684        3.2  0.127289   
4  California          39557045         253.906502        1.8  0.124635   

   pop_10-19  pop_20-29  pop_30-39  pop_40-49  pop_50-59  pop_60-69  \
0   0.128108   0.136099   0.124291   0.123013   0.133249   0.120537   
1   0.130210   0.155501   0.147626   0.117347   0.127204   0.109072   
2   0.132496   0.140854   0.129224   0.119039   0.120016   0.113492   
3   0.132222   0.135421   0.126671   0.119787   0.126968   0.115755   
4   0.128760   0.147575   0.143412   0.128138   0.126974   0.104040   

   pop_70-79   pop_80+  hospitalitzats  morts  contagiats  
0   0.075243  0.038527            1176     55       34763  
1 

In [ ]:
# IA (Incidència Acumulada)
df["IA"] = (df["contagiats"] / df["population_state"]) * 100000

# θ (Ocupació hospitalària %)
df["beds_real"] = (df["bedsTotal"] * df["population_state"]) / 1000
df["theta"] = (df["hospitalitzats"] / df["beds_real"]) * 100

# π (Taxa de mortalitat %) per 100000 habitants
df["pi"] = (df["morts"] / df["population_state"]) * 100000

# % població >65
df["pop_>65"] = df["pop_70-79"] + df["pop_80+"]

df["viatgers"] = df["IA"] > 100

In [ ]:
df.head()


,state,population_state,pop_density_state,bedsTotal,pop_0-9,pop_10-19,pop_20-29,pop_30-39,pop_40-49,pop_50-59,...,pop_80+,hospitalitzats,morts,contagiats,IA,beds_real,theta,pi,pop_>65,viatgers
0,Alabama,4887871,96.509389,3.1,0.120933,0.128108,0.136099,0.124291,0.123013,0.133249,...,0.038527,1176,55,34763,711.209441,15152.4001,7.761147,1.125234,0.113770,True
1,Alaska,737438,1.291523,2.2,0.142617,0.130210,0.155501,0.147626,0.117347,0.127204,...,0.019748,1509,368,33606,4557.128870,1622.3636,93.012442,49.902500,0.070424,True
2,Arizona,7171646,63.135855,1.9,0.123774,0.132496,0.140854,0.129224,0.119039,0.120016,...,0.040799,910,59,12534,174.771594,13626.1274,6.678346,0.822684,0.121105,True
3,Arkansas,3013825,57.919684,3.2,0.127289,0.132222,0.135421,0.126671,0.119787,0.126968,...,0.040010,1344,248,30127,999.626720,9644.2400,13.935779,8.228746,0.115888,True
4,California,39557045,253.906502,1.8,0.124635,0.128760,0.147575,0.143412,0.128138,0.126974,...,0.035566,1180,324,41397,104.651396,71202.6810,1.657241,0.819070,0.096466,True


In [52]:
defuncions = ctrl.Antecedent(np.arange(0, 501, 1), 'defuncions')  # Exemple: 0-500 morts
infectats = ctrl.Antecedent(np.arange(0, 100001, 1), 'infectats')  # Exemple: 0-100k infectats
hospitalitzats = ctrl.Antecedent(np.arange(0, 10001, 1), 'hospitalitzats')  # Exemple: 0-10k hospitalitzats


risc = ctrl.Consequent(np.arange(0, 11, 1), 'risc')

# Funcions d'afiliació (exemple)
defuncions.automf(3, names=['Baix', 'Mitjà', 'Alt'])
infectats.automf(3, names=['Baix', 'Mitjà', 'Alt'])
hospitalitzats.automf(3, names=['Baix', 'Mitjà', 'Alt'])

risc['Baix'] = fuzz.trimf(risc.universe, [0, 0, 5])
risc['Mitjà'] = fuzz.trimf(risc.universe, [0, 5, 10])
risc['Alt'] = fuzz.trimf(risc.universe, [5, 10, 10])

# Regles difuses (exemple)
regla1 = ctrl.Rule(defuncions['Alt'] | infectats['Alt'] | hospitalitzats['Alt'], risc['Alt'])
regla2 = ctrl.Rule(defuncions['Mitjà'] & infectats['Mitjà'], risc['Mitjà'])
regla3 = ctrl.Rule(defuncions['Baix'] & infectats['Baix'] & hospitalitzats['Baix'], risc['Baix'])

sistema_risc = ctrl.ControlSystem([regla1, regla2, regla3])
calculador_risc = ctrl.ControlSystemSimulation(sistema_risc)

In [53]:
resultats = []
for _, estat in df_combinat.iterrows():
    try:
        calculador_risc.input['defuncions'] = estat['morts']
        calculador_risc.input['infectats'] = estat['contagiats']
        calculador_risc.input['hospitalitzats'] = estat['hospitalitzats']
        
        calculador_risc.compute()
        resultats.append(calculador_risc.output['risc'])
    except Exception as e:
        print(f"Error a l'estat {estat['state']}: {e}")
        resultats.append(np.nan)  # Afegeix NaN si hi ha error

# Afegeix els resultats al DataFrame
df_combinat['risc'] = resultats
print(df_combinat[['state', 'risc']])

                   state      risc
0                Alabama  4.352421
1                 Alaska  5.533006
2                Arizona  3.479155
3               Arkansas  4.995570
4             California  5.223613
5               Colorado  4.491967
6            Connecticut  7.887089
7               Delaware  5.189115
8   District of Columbia  4.504773
9                Florida  3.048313
10               Georgia  5.319452
11                Hawaii  4.172460
12                 Idaho  5.085925
13              Illinois  6.403960
14               Indiana  4.143234
15                  Iowa  5.389974
16                Kansas  2.738277
17              Kentucky  4.992242
18             Louisiana  5.096328
19                 Maine  5.376812
20              Maryland  4.739319
21         Massachusetts  5.760706
22              Michigan  5.339552
23             Minnesota  2.647217
24           Mississippi  7.259238
25              Missouri  7.508470
26               Montana  6.931332
27              Nebr

In [54]:
df_demografic_original.columns

Index(['state', 'no_coverage', 'private_coverage', 'public_coverage',
       'labor_cov_diff', 'bedsState_local_government', 'bedsNon_profit',
       'bedsFor_profit', 'bedsTotal', 'population_state', 'pop_density_state',
       'pop_0-9', 'pop_10-19', 'pop_20-29', 'pop_30-39', 'pop_40-49',
       'pop_50-59', 'pop_60-69', 'pop_70-79', 'pop_80+', 'Low_SVI_CTGY',
       'Moderate_Low_SVI_CTGY', 'Moderate_High_SVI_CTGY', 'High_SVI_CTGY',
       'Metro', 'Non-metro'],
      dtype='object')

In [55]:
df_demografic.columns

Index(['state', 'population_state', 'pop_density_state', 'bedsTotal',
       'pop_0-9', 'pop_10-19', 'pop_20-29', 'pop_30-39', 'pop_40-49',
       'pop_50-59', 'pop_60-69', 'pop_70-79', 'pop_80+'],
      dtype='object')

In [56]:
df_combinat.columns

Index(['state', 'population_state', 'pop_density_state', 'bedsTotal',
       'pop_0-9', 'pop_10-19', 'pop_20-29', 'pop_30-39', 'pop_40-49',
       'pop_50-59', 'pop_60-69', 'pop_70-79', 'pop_80+', 'hospitalitzats',
       'morts', 'contagiats', 'IA', 'beds_real', 'theta', 'pi', 'pop_>65',
       'viatgers', 'risc'],
      dtype='object')

In [57]:
df_combinat.head()

,state,population_state,pop_density_state,bedsTotal,pop_0-9,pop_10-19,pop_20-29,pop_30-39,pop_40-49,pop_50-59,...,hospitalitzats,morts,contagiats,IA,beds_real,theta,pi,pop_>65,viatgers,risc
0,Alabama,4887871,96.509389,3.1,0.120933,0.128108,0.136099,0.124291,0.123013,0.133249,...,1176,55,34763,711.209441,15152.4001,7.761147,1.125234,0.113770,True,4.352421
1,Alaska,737438,1.291523,2.2,0.142617,0.130210,0.155501,0.147626,0.117347,0.127204,...,1509,368,33606,4557.128870,1622.3636,93.012442,49.902500,0.070424,True,5.533006
2,Arizona,7171646,63.135855,1.9,0.123774,0.132496,0.140854,0.129224,0.119039,0.120016,...,910,59,12534,174.771594,13626.1274,6.678346,0.822684,0.121105,True,3.479155
3,Arkansas,3013825,57.919684,3.2,0.127289,0.132222,0.135421,0.126671,0.119787,0.126968,...,1344,248,30127,999.626720,9644.2400,13.935779,8.228746,0.115888,True,4.995570
4,California,39557045,253.906502,1.8,0.124635,0.128760,0.147575,0.143412,0.128138,0.126974,...,1180,324,41397,104.651396,71202.6810,1.657241,0.819070,0.096466,True,5.223613


-----------------------------------------

## Aplicació de regles de coneixement difús 

In [58]:
'''
def classifica_risc(row):
    if row["Rt"] > 1.5 and row["IA"] > 150 and row["theta"] > 80:
        return "Crític"
    elif row["Rt"] > 1.2 and (row["IA"] > 100 or row["pi"] > 1.5) and row["theta"] > 70:
        return "Alt"
    elif row["IA"] > 50 and row["positivity_rate"] > 5:
        return "Moderat"
    else:
        return "Baix"

df_combinat["nivell_risc"] = df_combinat.apply(classifica_risc, axis=1)

# Mostrem les dades clau resultants
print(df_combinat[["state", "IA", "Rt", "theta", "pi", "positivity_rate", "pop_>65", "nivell_risc"]].head())
'''

'\ndef classifica_risc(row):\n    if row["Rt"] > 1.5 and row["IA"] > 150 and row["theta"] > 80:\n        return "Crític"\n    elif row["Rt"] > 1.2 and (row["IA"] > 100 or row["pi"] > 1.5) and row["theta"] > 70:\n        return "Alt"\n    elif row["IA"] > 50 and row["positivity_rate"] > 5:\n        return "Moderat"\n    else:\n        return "Baix"\n\ndf_combinat["nivell_risc"] = df_combinat.apply(classifica_risc, axis=1)\n\n# Mostrem les dades clau resultants\nprint(df_combinat[["state", "IA", "Rt", "theta", "pi", "positivity_rate", "pop_>65", "nivell_risc"]].head())\n'